In [ ]:
import torch
from transformers import RobertaModel, RobertaTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model

In [ ]:
base_model = 'roberta-base'

In [ ]:
import pandas as pd
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_loc = "/content/drive/MyDrive/kmeans_binary_labeled_helpfulness_data_with_modified_review.csv"
df = pd.read_csv(dataset_loc)

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(base_model)
def tokenize_function(examples):
    return tokenizer(examples['modified_review'], padding='max_length', truncation=True, max_length=256)

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(df['modified_review'], df['kmeans_label'], test_size=0.2, random_state=42)

In [ ]:
train_labels = train_labels.tolist()
test_labels = test_labels.tolist()

In [ ]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding='max_length', max_length=256)
test_encodings = tokenizer(list(test_texts), truncation=True, padding='max_length', max_length=256)

In [ ]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = ReviewDataset(train_encodings, train_labels)
test_dataset = ReviewDataset(test_encodings, test_labels)

In [ ]:
training_args = TrainingArguments(
    output_dir='./',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=2)


trainer = Trainer(
    model=model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kars1 (kars1-university-of-nevada-las-vegas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.686400
20,0.702600
30,0.702600
40,0.692700
50,0.682800
60,0.698900
70,0.676700
80,0.674300
90,0.736000
100,0.669000


TrainOutput(global_step=12213, training_loss=0.6943177707968536, metrics={'train_runtime': 5336.692, 'train_samples_per_second': 18.307, 'train_steps_per_second': 2.288, 'total_flos': 1.285271194328064e+16, 'train_loss': 0.6943177707968536, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import classification_report
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(axis=-1)

# Evaluation report
print(classification_report(test_labels, preds))

              precision    recall  f1-score   support

           0       0.51      1.00      0.67      4139
           1       0.00      0.00      0.00      4003

    accuracy                           0.51      8142
   macro avg       0.25      0.50      0.34      8142
weighted avg       0.26      0.51      0.34      8142



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
